### Import Libraries

In [270]:
import pandas as pd
import numpy as np
import random
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import classification_report

import numpy as np, warnings
warnings.filterwarnings("ignore") #Ignore warnings

### Ingest and process Data

In [271]:
datatrain = pd.read_csv('data/yeast/yeast-train.csv')
datatest = pd.read_csv('data/yeast/yeast-test.csv')

datatrain.loc[datatrain['Class1'] =="b'0'", 'Class1'] = 0
datatrain.loc[datatrain['Class1'] =="b'1'", 'Class1'] = 1
datatest.loc[datatest['Class1'] =="b'0'", 'Class1'] = 0
datatest.loc[datatest['Class1'] =="b'1'", 'Class1'] = 1

dataset = pd.concat([datatrain, datatest], ignore_index=True)
# dataset.head()

In [272]:
classNum=1
y_class = 102 + classNum
X = dataset.iloc[:,:103].values
y = dataset.iloc[:,y_class:y_class+1].values
# X_train = datatrain.iloc[:,:103].values
# y_train = datatrain.iloc[:,y_class:y_class+1].values
# X_test = datatest.iloc[:,:103].values
# y_test = datatest.iloc[:,y_class:y_class+1].values


In [273]:
random.seed(0)

In [275]:
#dataset['Class1'].value_counts()
# type(X)

In [276]:
# y=np_utils.to_categorical(y, num_classes = 2)
X_train,X_test, y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=0)


In [277]:
print(X_train.shape)
print(X_test.shape)

(1933, 103)
(484, 103)


In [278]:
d = pd.read_csv('relevance.csv')

In [279]:
d.drop(['columns'],axis=1,inplace=True)

In [280]:
# d=d.abs()
means= d.mean(axis=1)

In [281]:
means.abs().sort_values()

59     0.000011
21     0.000059
37     0.000196
91     0.000344
18     0.000389
92     0.000410
7      0.000447
46     0.000527
28     0.000597
26     0.000640
70     0.000659
66     0.000668
94     0.000668
56     0.000682
12     0.000685
84     0.000737
96     0.000773
3      0.000789
49     0.000843
44     0.000889
54     0.000962
63     0.001014
50     0.001061
32     0.001074
60     0.001123
31     0.001134
29     0.001158
71     0.001174
99     0.001185
45     0.001231
         ...   
79     0.002416
16     0.002502
53     0.002517
81     0.002610
88     0.002622
67     0.002651
19     0.002706
101    0.002811
65     0.002836
20     0.002857
75     0.002907
25     0.002932
36     0.002970
68     0.003005
40     0.003006
4      0.003006
13     0.003023
82     0.003085
51     0.003138
14     0.003141
17     0.003175
93     0.003226
1      0.003286
57     0.003619
95     0.003774
80     0.003826
77     0.003962
2      0.004151
87     0.004446
102    0.005696
Length: 103, dtype: floa

In [300]:
toRemove=[]

In [301]:
for i in range(21):
    x=np.where(means.values == np.amin(means.values))
    means.values[x[0][0]]=1
    toRemove.append(x[0][0])

In [302]:
print(toRemove,end=', ')

[50, 32, 60, 31, 29, 71, 99, 45, 0, 89, 48, 34, 62, 11, 85, 61, 73, 41, 72, 83, 24], 

In [303]:
X_train=np.delete(X_train,toRemove,1)
X_test=np.delete(X_test,toRemove,1)

In [289]:
a = [el/100 for el in range(0,100) ]

In [292]:
X_train.shape,X_test.shape,trainY.shape,testY.shape,y_train.shape,y_test.shape

((1933, 82), (484, 82), (1933, 2), (484, 2), (1933, 1), (484, 1))

### Model

#### XGBoost

In [262]:
# mxa=0
# for i in a:
xgbclassifier = XGBClassifier(learning_rate=0.27)
xgbclassifier.fit(X_train, y_train)
y_pred = xgbclassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
#     if accuracy>mxa:
#         mxa=accuracy
#         print(mxa,i+0.01)

Accuracy: 78.93%


In [10]:
# save model to file
pickle.dump(xgbclassifier, open("models/xgbModelplain.pickle.dat", "wb"))
print("Model Saved...\n")

score = xgbclassifier.score(X_test,y_test)
print("Model Accuracy: " + str(score) + "\n")
predY = xgbclassifier.predict(X_test)

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, predY))

print("\nClassification Report:\n")
print(classification_report(y_test, predY))

Model Saved...

Model Accuracy: 0.7995867768595041

Confusion Matrix:

[[307  23]
 [ 74  80]]

Classification Report:

              precision    recall  f1-score   support

           0       0.81      0.93      0.86       330
           1       0.78      0.52      0.62       154

   micro avg       0.80      0.80      0.80       484
   macro avg       0.79      0.72      0.74       484
weighted avg       0.80      0.80      0.79       484



#### SVM

In [265]:
from sklearn.svm import SVC

svmclassifier = SVC(kernel='linear' ,random_state=0)
svmclassifier.fit(X_train,y_train)
y_pred_svm = svmclassifier.predict(X_test)

cm = confusion_matrix(y_test,y_pred_svm)
print(cm)
accuracy=accuracy_score(y_test, y_pred_svm)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[[313  17]
 [ 87  67]]
Accuracy: 78.51%


In [12]:
# save model to file
pickle.dump(svmclassifier, open("models/svmModelplain.pickle.dat", "wb"))
print("Model Saved...\n")

score = svmclassifier.score(X_test,y_test)
print("Model Accuracy: " + str(score) + "\n")
y_pred_svm = svmclassifier.predict(X_test)

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_svm))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_svm))

Model Saved...

Model Accuracy: 0.7851239669421488

Confusion Matrix:

[[311  19]
 [ 85  69]]

Classification Report:

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       330
           1       0.78      0.45      0.57       154

   micro avg       0.79      0.79      0.79       484
   macro avg       0.78      0.70      0.71       484
weighted avg       0.78      0.79      0.77       484



### Random Forest

In [264]:
from sklearn.ensemble import RandomForestClassifier
# mxa = 0
# for i in range(1,50):
rfclassifier = RandomForestClassifier(n_estimators=45,criterion='entropy',random_state=0)
rfclassifier.fit(X_train,y_train)
y_pred_randForest = rfclassifier.predict(X_test)
accuracy=accuracy_score(y_test, y_pred_randForest)
#     if accuracy>mxa:
#         mxa=accuracy
#         print(accuracy,(i*5))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 78.10%


In [14]:
# save model to file
pickle.dump(rfclassifier, open("models/rfModelplain.pickle.dat", "wb"))
print("Model Saved...\n")

score = rfclassifier.score(X_test,y_test)
print("Model Accuracy: " + str(score) + "\n")
y_pred_randForest = rfclassifier.predict(X_test)

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_randForest))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_randForest))

Model Saved...

Model Accuracy: 0.7933884297520661

Confusion Matrix:

[[320  10]
 [ 90  64]]

Classification Report:

              precision    recall  f1-score   support

           0       0.78      0.97      0.86       330
           1       0.86      0.42      0.56       154

   micro avg       0.79      0.79      0.79       484
   macro avg       0.82      0.69      0.71       484
weighted avg       0.81      0.79      0.77       484



### Neural Net

In [8]:
# Packages for training model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

# Packages for explanation
from keras import backend as K
from keras.models import Model
from deepexplain.tensorflow import DeepExplain
from IPython.display import display, HTML
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [223]:
sgdOptimizer = 'adam'
lossFun='categorical_crossentropy'
finalLayerActivation = 'softmax'
batchSize=25
numEpochs = 500
nb_classes = 2



In [304]:
for i,v in enumerate(toRemove):
    toRemove[i]=toRemove[i]='Att'+str(v+1)

In [306]:
print(toRemove,end=', ')

['Att51', 'Att33', 'Att61', 'Att32', 'Att30', 'Att72', 'Att100', 'Att46', 'Att1', 'Att90', 'Att49', 'Att35', 'Att63', 'Att12', 'Att86', 'Att62', 'Att74', 'Att42', 'Att73', 'Att84', 'Att25'], 

In [307]:
# Define training data
features = ['Att1', 'Att2', 'Att3', 'Att4', 'Att5', 'Att6', 'Att7', 'Att8', 'Att9', 
            'Att10', 'Att11', 'Att12', 'Att13', 'Att14', 'Att15', 'Att16', 'Att17', 
            'Att18', 'Att19', 'Att20', 'Att21', 'Att22', 'Att23', 'Att24', 'Att25', 
            'Att26', 'Att27', 'Att28', 'Att29', 'Att30', 'Att31', 'Att32', 'Att33', 
            'Att34', 'Att35', 'Att36', 'Att37', 'Att38', 'Att39', 'Att40', 'Att41', 
            'Att42', 'Att43', 'Att44', 'Att45', 'Att46', 'Att47', 'Att48', 'Att49', 
            'Att50', 'Att51', 'Att52', 'Att53', 'Att54', 'Att55', 'Att56', 'Att57', 
            'Att58', 'Att59', 'Att60', 'Att61', 'Att62', 'Att63', 'Att64', 'Att65', 
            'Att66', 'Att67', 'Att68', 'Att69', 'Att70', 'Att71', 'Att72', 'Att73', 
            'Att74', 'Att75', 'Att76', 'Att77', 'Att78', 'Att79', 'Att80', 'Att81', 
            'Att82', 'Att83', 'Att84', 'Att85', 'Att86', 'Att87', 'Att88', 'Att89', 
            'Att90', 'Att91', 'Att92', 'Att93', 'Att94', 'Att95', 'Att96', 'Att97', 
            'Att98', 'Att99', 'Att100', 'Att101', 'Att102', 'Att103']

In [308]:
for i in toRemove:
    if i in features:
        features.remove(i)

In [311]:
len(features)

82

In [256]:
x_train = dataset[features]
inputDim = len(features)
trainX = x_train

# y_train = dataset['Class1']
# trainY = np_utils.to_categorical(y_train, num_classes = nb_classes)
trainY = np_utils.to_categorical(y_train, num_classes = nb_classes)
testY = np_utils.to_categorical(y_test, num_classes = nb_classes)

In [257]:

# Define model
model = Sequential()
model.add(BatchNormalization(input_shape=(inputDim,)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(nb_classes, activation=finalLayerActivation))
model.compile(loss=lossFun, optimizer=sgdOptimizer, metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_13 (Batc (None, 82)                328       
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2656      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 32)                128       
_________________________________________________________________
dense_18 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 32)                128       
__________

In [266]:
X_train.shape,X_test.shape,trainY.shape,testY.shape

((1933, 82), (484, 82), (1933, 2), (484, 2))

In [261]:
model.fit(X_train,trainY,validation_data=(X_test,testY) ,batch_size=batchSize, epochs=numEpochs, verbose=1)
trainScore = model.evaluate(X_train,trainY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], trainScore[1]*100))
testScore = model.evaluate(X_test,testY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], testScore[1]*100))

Train on 1933 samples, validate on 484 samples
Epoch 1/500
1933/1933 [==============================] - 1s 336us/step - loss: 0.3296 - acc: 0.8557 - val_loss: 0.5987 - val_acc: 0.7562
Epoch 2/500
1933/1933 [==============================] - 1s 299us/step - loss: 0.3345 - acc: 0.8557 - val_loss: 0.5758 - val_acc: 0.7603
Epoch 3/500
1933/1933 [==============================] - 1s 292us/step - loss: 0.3390 - acc: 0.8520 - val_loss: 0.5917 - val_acc: 0.7500
Epoch 4/500
1933/1933 [==============================] - 1s 313us/step - loss: 0.3461 - acc: 0.8541 - val_loss: 0.5589 - val_acc: 0.7562
Epoch 5/500
1933/1933 [==============================] - 1s 300us/step - loss: 0.3528 - acc: 0.8453 - val_loss: 0.5739 - val_acc: 0.7459
Epoch 6/500
1933/1933 [==============================] - 1s 308us/step - loss: 0.3444 - acc: 0.8562 - val_loss: 0.5727 - val_acc: 0.7479
Epoch 7/500
1933/1933 [==============================] - 1s 306us/step - loss: 0.3455 - acc: 0.8469 - val_loss: 0.5873 - val_acc: 0

1933/1933 [==============================] - 1s 289us/step - loss: 0.3400 - acc: 0.8469 - val_loss: 0.5867 - val_acc: 0.7603
Epoch 61/500
1933/1933 [==============================] - 1s 278us/step - loss: 0.3278 - acc: 0.8614 - val_loss: 0.6062 - val_acc: 0.7603
Epoch 62/500
1933/1933 [==============================] - 1s 290us/step - loss: 0.3274 - acc: 0.8588 - val_loss: 0.5878 - val_acc: 0.7645
Epoch 63/500
1933/1933 [==============================] - 1s 280us/step - loss: 0.3331 - acc: 0.8526 - val_loss: 0.5785 - val_acc: 0.7583
Epoch 64/500
1933/1933 [==============================] - 1s 295us/step - loss: 0.3214 - acc: 0.8551 - val_loss: 0.6134 - val_acc: 0.7562
Epoch 65/500
1933/1933 [==============================] - 1s 292us/step - loss: 0.3439 - acc: 0.8531 - val_loss: 0.6069 - val_acc: 0.7479
Epoch 66/500
1933/1933 [==============================] - 1s 286us/step - loss: 0.3449 - acc: 0.8484 - val_loss: 0.5959 - val_acc: 0.7541
Epoch 67/500
1933/1933 [=======================

1933/1933 [==============================] - 1s 289us/step - loss: 0.3309 - acc: 0.8474 - val_loss: 0.6605 - val_acc: 0.7748
Epoch 179/500
1933/1933 [==============================] - 1s 292us/step - loss: 0.3088 - acc: 0.8681 - val_loss: 0.6531 - val_acc: 0.7727
Epoch 180/500
1933/1933 [==============================] - 1s 289us/step - loss: 0.3307 - acc: 0.8536 - val_loss: 0.6240 - val_acc: 0.7789
Epoch 181/500
1933/1933 [==============================] - 1s 294us/step - loss: 0.3243 - acc: 0.8608 - val_loss: 0.6130 - val_acc: 0.7748
Epoch 182/500
1933/1933 [==============================] - 1s 296us/step - loss: 0.3188 - acc: 0.8598 - val_loss: 0.6145 - val_acc: 0.7789
Epoch 183/500
1933/1933 [==============================] - 1s 294us/step - loss: 0.3258 - acc: 0.8577 - val_loss: 0.6279 - val_acc: 0.7727
Epoch 184/500
1933/1933 [==============================] - 1s 286us/step - loss: 0.3246 - acc: 0.8577 - val_loss: 0.6280 - val_acc: 0.7707
Epoch 185/500
1933/1933 [================

1933/1933 [==============================] - 1s 298us/step - loss: 0.3427 - acc: 0.8417 - val_loss: 0.6120 - val_acc: 0.7521
Epoch 238/500
1933/1933 [==============================] - 1s 311us/step - loss: 0.3340 - acc: 0.8520 - val_loss: 0.6094 - val_acc: 0.7479
Epoch 239/500
1933/1933 [==============================] - 1s 290us/step - loss: 0.3209 - acc: 0.8577 - val_loss: 0.6318 - val_acc: 0.7521
Epoch 240/500
1933/1933 [==============================] - 1s 288us/step - loss: 0.3381 - acc: 0.8469 - val_loss: 0.6276 - val_acc: 0.7583
Epoch 241/500
1933/1933 [==============================] - 1s 314us/step - loss: 0.2987 - acc: 0.8629 - val_loss: 0.6757 - val_acc: 0.7583
Epoch 242/500
1933/1933 [==============================] - 1s 305us/step - loss: 0.3438 - acc: 0.8464 - val_loss: 0.6320 - val_acc: 0.7500
Epoch 243/500
1933/1933 [==============================] - 1s 306us/step - loss: 0.3344 - acc: 0.8510 - val_loss: 0.6203 - val_acc: 0.7562
Epoch 244/500
1933/1933 [================

1933/1933 [==============================] - 1s 307us/step - loss: 0.3436 - acc: 0.8458 - val_loss: 0.6003 - val_acc: 0.7748
Epoch 297/500
1933/1933 [==============================] - 1s 279us/step - loss: 0.3357 - acc: 0.8479 - val_loss: 0.6206 - val_acc: 0.7748
Epoch 298/500
1933/1933 [==============================] - 1s 288us/step - loss: 0.3074 - acc: 0.8691 - val_loss: 0.6154 - val_acc: 0.7707
Epoch 299/500
1933/1933 [==============================] - 1s 298us/step - loss: 0.3208 - acc: 0.8546 - val_loss: 0.6374 - val_acc: 0.7789
Epoch 300/500
1933/1933 [==============================] - 1s 305us/step - loss: 0.3425 - acc: 0.8432 - val_loss: 0.6132 - val_acc: 0.7727
Epoch 301/500
1933/1933 [==============================] - 1s 291us/step - loss: 0.3258 - acc: 0.8608 - val_loss: 0.6303 - val_acc: 0.7727
Epoch 302/500
1933/1933 [==============================] - 1s 320us/step - loss: 0.3336 - acc: 0.8526 - val_loss: 0.6118 - val_acc: 0.7686
Epoch 303/500
1933/1933 [================

1933/1933 [==============================] - 1s 294us/step - loss: 0.3159 - acc: 0.8634 - val_loss: 0.6253 - val_acc: 0.7541
Epoch 356/500
1933/1933 [==============================] - 1s 294us/step - loss: 0.3315 - acc: 0.8474 - val_loss: 0.6019 - val_acc: 0.7665
Epoch 357/500
1933/1933 [==============================] - 1s 285us/step - loss: 0.2978 - acc: 0.8624 - val_loss: 0.6439 - val_acc: 0.7686
Epoch 358/500
1933/1933 [==============================] - 1s 283us/step - loss: 0.3265 - acc: 0.8505 - val_loss: 0.6039 - val_acc: 0.7624
Epoch 359/500
1933/1933 [==============================] - 1s 300us/step - loss: 0.3309 - acc: 0.8541 - val_loss: 0.6007 - val_acc: 0.7541
Epoch 360/500
1933/1933 [==============================] - 1s 292us/step - loss: 0.3145 - acc: 0.8691 - val_loss: 0.6351 - val_acc: 0.7583
Epoch 361/500
1933/1933 [==============================] - 1s 298us/step - loss: 0.3048 - acc: 0.8639 - val_loss: 0.6303 - val_acc: 0.7624
Epoch 362/500
1933/1933 [================

1933/1933 [==============================] - 1s 311us/step - loss: 0.3261 - acc: 0.8520 - val_loss: 0.6280 - val_acc: 0.7417
Epoch 415/500
1933/1933 [==============================] - 1s 282us/step - loss: 0.3235 - acc: 0.8546 - val_loss: 0.6128 - val_acc: 0.7500
Epoch 416/500
1933/1933 [==============================] - 1s 298us/step - loss: 0.3409 - acc: 0.8500 - val_loss: 0.6111 - val_acc: 0.7500
Epoch 417/500
1933/1933 [==============================] - 1s 288us/step - loss: 0.3090 - acc: 0.8619 - val_loss: 0.6626 - val_acc: 0.7500
Epoch 418/500
1933/1933 [==============================] - 1s 300us/step - loss: 0.3301 - acc: 0.8572 - val_loss: 0.6330 - val_acc: 0.7521
Epoch 419/500
1933/1933 [==============================] - 1s 294us/step - loss: 0.3346 - acc: 0.8500 - val_loss: 0.6439 - val_acc: 0.7541
Epoch 420/500
1933/1933 [==============================] - 1s 290us/step - loss: 0.3288 - acc: 0.8520 - val_loss: 0.6575 - val_acc: 0.7459
Epoch 421/500
1933/1933 [================

1933/1933 [==============================] - 1s 299us/step - loss: 0.3214 - acc: 0.8567 - val_loss: 0.6196 - val_acc: 0.7603
Epoch 474/500
1933/1933 [==============================] - 1s 287us/step - loss: 0.3270 - acc: 0.8598 - val_loss: 0.6151 - val_acc: 0.7603
Epoch 475/500
1933/1933 [==============================] - 1s 300us/step - loss: 0.3284 - acc: 0.8593 - val_loss: 0.6387 - val_acc: 0.7665
Epoch 476/500
1933/1933 [==============================] - 1s 299us/step - loss: 0.3318 - acc: 0.8464 - val_loss: 0.6437 - val_acc: 0.7603
Epoch 477/500
1933/1933 [==============================] - 1s 295us/step - loss: 0.3349 - acc: 0.8510 - val_loss: 0.6260 - val_acc: 0.7686
Epoch 478/500
1933/1933 [==============================] - 1s 293us/step - loss: 0.3512 - acc: 0.8495 - val_loss: 0.6077 - val_acc: 0.7707
Epoch 479/500
1933/1933 [==============================] - 1s 296us/step - loss: 0.3157 - acc: 0.8500 - val_loss: 0.6435 - val_acc: 0.7686
Epoch 480/500
1933/1933 [================

In [78]:
predY = model.predict(X_train)
print("Confusion Matrix:\n")

prediction = model.predict_classes(X_train, verbose=0)
print(confusion_matrix(y_train, prediction))

print("\nClassification Report:\n")
print(classification_report(y_train, prediction))

Confusion Matrix:

[[1311   14]
 [  73  535]]

Classification Report:

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1325
           1       0.97      0.88      0.92       608

   micro avg       0.95      0.95      0.95      1933
   macro avg       0.96      0.93      0.95      1933
weighted avg       0.96      0.95      0.95      1933

